lunana  
last update 2022 05 31

**This notebook is a labo note for sentence bert.**

![https://1.bp.blogspot.com/-6rJFNlG1dDM/WerKkgZKwSI/AAAAAAABHpM/cKdrgn1ZSiMVBOmIkhdJ-dkrnpJ8Q1SEQCLcBGAs/s400/computer_hakui_doctor_woman.png](https://1.bp.blogspot.com/-6rJFNlG1dDM/WerKkgZKwSI/AAAAAAABHpM/cKdrgn1ZSiMVBOmIkhdJ-dkrnpJ8Q1SEQCLcBGAs/s400/computer_hakui_doctor_woman.png)

# EDA  
**Foursquare EDA+sentence bert 実況**  
https://www.kaggle.com/code/lunapandachan/foursquare-eda-sentence-bert

* [**V0 Prepare labo & load data**](#V0-Prepare-labo-&-load-data)
* [**V1 sentence transformers**](#V1-sentence-transformers)  
* [**V2 vector**](#V2-vector)  
* [**V3 Labeled train**](#V3-Labeled-train)  
* [**V4 sentence transformers original**](#V4-sentence-transformers-original)  
* [**V5 next labeled train**](#V5-next-labeled-train)  
* [**V6 next labeled train epoch5**](#V6-next-labeled-train-epoch5)
* [**V7 train data**](#V7-train-data)

# Define

## switches

In [ ]:
IS_Fig_Only=True
IS_mini_DS=False
IS_V0_make_mini_DS=False
IS_V1_train=False
IS_V2_eval=False
IS_V2_encode=False
IS_V3_train=False
IS_V4_train=False
IS_V5_train=False
IS_V6_train=False
IS_V7_eval=False

## global values

In [ ]:
input_path='../input/foursquare-location-matching'
output_path='.'
ds_path='../input/foursquare-s-bert-train'
ds2_path='../input/fourauquare-ds2'
ds_fig_path='../input/foursquare-ds-fig'

## common CFG

In [ ]:
class CommonCFG:
    colab_input_path='/content/drive/MyDrive/kaggle/Foursquare/input'
    colab_output_path='/content/drive/MyDrive/kaggle/Foursquare/output'
    colab_key_path='/content/drive/MyDrive/kaggle'
    colab_ds_path='/content/drive/MyDrive/kaggle/Foursquare/ds'
    colab_ds2_path='/content/drive/MyDrive/kaggle/Foursquare/ds2'
    colab_ds_fig_path='/content/drive/MyDrive/kaggle/Foursquare/ds-fig'
   

## CFG

In [ ]:
class CFG:
    pairs='pairs.csv'
    train='train.csv'
    s_pairs='s_pairs.csv'
    s_train='s_train.csv'
    sup_train='sup_train.csv'
    s_size=5000
    device='cuda'
    n_sample=40000
    seed=42

# V0 Prepare labo & load data

In [ ]:
import os
IS_COLAB = not os.path.exists('/kaggle/input')
print(IS_COLAB) 

In [ ]:
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install kaggle
    !mkdir -p ~/.kaggle/ && cp $CommonCFG.colab_key_path/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    input_path=CommonCFG.colab_input_path
    output_path=CommonCFG.colab_output_path
    ds_path=CommonCFG.colab_ds_path
    ds2_path=CommonCFG.colab_ds2_path
    ds_fig_path=CommonCFG.colab_ds_fig_path

## common modules

In [ ]:
import pandas as pd
import numpy as np
import pickle
import gc
from IPython.display import Image
import matplotlib.pyplot as plt

## making small datasets

In [ ]:
if IS_V0_make_mini_DS:
    df_pairs=pd.read_csv(input_path+'/'+CFG.pairs)
    df_pairs_s=df_pairs[:CFG.s_size].copy()
    df_pairs_s.to_csv(output_path+'/'+CFG.s_pairs, index=False)
    print(len(df_pairs_s))
    print(len(df_pairs_s[df_pairs_s['match']==True]))

    lst_id=np.concatenate([df_pairs_s['id_1'].values,df_pairs_s['id_2'].values])
    lst_id=pd.Series(lst_id).unique()
    lst_id=pd.DataFrame(lst_id,columns=['id'])
    print(len(lst_id))
    df_train=pd.read_csv(input_path+'/'+CFG.train)
    df_train_s=lst_id.merge(df_train,on=['id'],how='left')
    df_train_s.to_csv(output_path+'/'+CFG.s_train, index=False)

    df_train=df_train_s
    df_pairs=df_pairs_s
    

In [ ]:
if IS_mini_DS:
    if  not IS_V0_make_mini_DS:
        df_train=pd.read_csv(ds_path+'/'+CFG.s_train)
        df_pairs=pd.read_csv(ds_path+'/'+CFG.s_pairs)


## common function

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pylab as plt

def make_fig(df,embed,dict_poi,save_fn='fig.png'):
    tsne = TSNE(
      n_components=2, random_state=0
    )
    # SCDVの次元削減をする
    tsne2=tsne.fit_transform(embed)
    colors = [cm.jet(dict_poi[i]/(len(dict_poi)-1)) for i in df['point_of_interest']]
    fig=plt.figure(figsize=(10, 10))
    plt.scatter(tsne2[:, 0], tsne2[:, 1], c=colors)
    fig.savefig(save_fn)

# V1 sentence transformers

## install modules

In [ ]:
!pip install -U sentence-transformers
import sentence_transformers

In [ ]:
model = sentence_transformers.SentenceTransformer('xlm-roberta-base',device=CFG.device)

## dataset

In [ ]:
if not IS_Fig_Only:
    df_train=pd.read_csv(input_path+'/'+CFG.train)
    df_pairs=pd.read_csv(input_path+'/'+CFG.pairs)
    df_train_g=df_train.groupby('point_of_interest').size()
    df_train_g_small=df_train_g.sort_values(ascending=False)[:12].copy()
    print(df_train_g_small)
    df_eval_idx=df_train_g_small.keys()
    print('df_evall_idx: ',df_eval_idx)
    df_eval=df_train[df_train['point_of_interest'].isin(df_eval_idx)].copy()
    print(len(df_eval))

In [ ]:
if not IS_Fig_Only:
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    print(dict_eval)
    df=df_eval.fillna('').reset_index(drop=True)
    df['text'] = df['latitude'].astype(str)+ '[SEP]'+df['longitude'].astype(str)+ '[SEP]'+df['name'] + '[SEP]' + df['address'] + '[SEP]' + df['city'] + '[SEP]' + df['state'] + '[SEP]' + df['country'] + '[SEP]' + df['url'] + '[SEP]' + df['categories'] 

## pretrained  eval

In [ ]:
if not IS_Fig_Only:
    model = sentence_transformers.SentenceTransformer('xlm-roberta-base',device=CFG.device)
    sentences = df['text'].tolist()
    embeddings = model.encode(sentences)

In [ ]:
if not IS_Fig_Only:
    print("Sentence:", sentences[0])
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())

### figure

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig1pre.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    make_fig(df,embeddings,dict_eval,save_fn=output_path+'/fig1pre.png')

**学習前は複数の色が混ざっている箇所があります。**  

**Before learning, there are places where multiple colors are mixed.**

## Train

### datasets

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample,losses
from torch.utils.data import DataLoader
from tqdm import tqdm

if IS_V1_train:
    train_examples = []

    df_p=df_pairs.fillna('')
    df_p['text_1']= df_p['latitude_1'].astype(str)+ '[SEP]'+df_p['longitude_1'].astype(str)+ '[SEP]'+df_p['name_1'] + '[SEP]' + df_p['address_1'] + '[SEP]' + df_p['city_1'] + '[SEP]' + df_p['state_1'] + '[SEP]' + df_p['country_1'] + '[SEP]' + df_p['url_1'] + '[SEP]' + df_p['categories_1'] 
    df_p['text_2']=  df_p['latitude_2'].astype(str)+ '[SEP]'+df_p['longitude_2'].astype(str)+ '[SEP]'+df_p['name_2'] + '[SEP]' + df_p['address_2'] + '[SEP]' + df_p['city_2'] + '[SEP]' + df_p['state_2'] + '[SEP]' + df_p['country_2'] + '[SEP]' + df_p['url_2'] + '[SEP]' + df_p['categories_2'] 

    l_df=len(df_p)

    for idx in tqdm(range(l_df)):
        tof=-0
        if df_p.loc[idx,'match']:
            tof=1
        train_examples.append(InputExample(texts=[df_p.loc[idx,'text_1'],df_p.loc[idx,'text_2']], label=tof))

### train

In [ ]:
if IS_V1_train:
    model = sentence_transformers.SentenceTransformer('xlm-roberta-base',device=CFG.device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    #train_loss = losses.CosineSimilarityLoss(model)
    train_loss = losses.ContrastiveLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

    with open(output_path+'/xlm-roberta-base.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    if not IS_Fig_Only:
        with open(ds_path+'/xlm-roberta-base.pkl', 'rb') as f:
            model=pickle.load( f )

In [ ]:
if not IS_Fig_Only:
    embeddings = model.encode(sentences)
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())

### post traned fig

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig1post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    make_fig(df,embeddings,dict_eval,save_fn=output_path+'/fig1post.png')

**学習前に比べ、色ごとにまとまっています。**  

**Compared to before learning, each color is organized.**

# V2 vector 

## random sampling

In [ ]:
if IS_V2_eval:
    df_eval=df_train.sample(n=CFG.n_sample, random_state=CFG.seed)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    np_eval=np.array(df_eval_g)
    a=np.linalg.norm(np_eval, ord=2)
    print(int(a*a))

## eval

In [ ]:
if IS_V2_eval:
    df=df_eval.fillna('').reset_index(drop=True)
    df['text'] = df['latitude'].astype(str)+ '[SEP]'+df['longitude'].astype(str)+ '[SEP]'+df['name'] + '[SEP]' + df['address'] + '[SEP]' + df['city'] + '[SEP]' + df['state'] + '[SEP]' + df['country'] + '[SEP]' + df['url'] + '[SEP]' + df['categories'] 

In [ ]:
if IS_V2_eval:
    sentences = df['text'].tolist()
    if IS_V2_encode:
        embeddings=model.encode(sentences)
        with open(output_path+'/eval-emb.pkl', 'wb') as f:
            pickle.dump(embeddings, f)
    else:
        with open(ds_path+'/eval-emb.pkl', 'rb') as f:
            embeddings=pickle.load(f)

## embeddings normalize

In [ ]:
if IS_V2_eval:
    emb_poi=df_eval['point_of_interest'].values
    emb=np.array(embeddings)
    del embeddings
    gc.collect()
    emb_n=np.array([emb[i]/np.linalg.norm(emb[i], ord=2) for i in range(len(emb))])
    del emb
    gc.collect()

## cosine similarity

In [ ]:
if IS_V2_eval:
    list_p=np.zeros(5001)
    list_n=np.zeros(5001)

    for batch in tqdm(range((len(emb_n)+1000)//1000)):
        start_p=batch*1000
        batch_emb=emb_n[start_p:start_p+1000]
        c_s=np.dot(emb_n,batch_emb.T).T
        for idx in range(len(c_s)):
            over_c_s=np.where(c_s[idx]>0.95)
            for idx2 in range(len(over_c_s[0])):
                _id=over_c_s[0][idx2]
                _p=int((c_s[idx][_id]-0.95)*10000)
                if start_p+idx==_id:
                    list_p[5000]+=1
                else:
                    if emb_poi[start_p+idx]==emb_poi[_id]:
                        list_p[_p]+=1
                    else:
                        list_n[_p]+=1

In [ ]:
if IS_V2_eval:
    total_p=list_p.sum()
    best_th=0
    best_score=0.0
    for i in range(5000):
        _th=1.0-i*0.00001
        _num_p=list_p[5000-i:5001].sum()
        _num_n=list_n[5000-i:5001].sum()
        _score=_num_p/(_num_n+total_p)
        if _score>best_score:
            best_score=_score
            best_th=_th
#    print(_th)
#    print(_num_p)
#    print(_num_n)
#    print(_score)
    print('best_score:',best_score)
    print('best_th:',best_th)

# V3 Labeled train

## common functions

In [ ]:
def make_text(df):
    df['text'] = df['latitude'].astype(str)+ '[SEP]'+df['longitude'].astype(str)+ '[SEP]'+df['name'] + '[SEP]' + df['address'] + '[SEP]' + df['city'] + '[SEP]' + df['state'] + '[SEP]' + df['country'] + '[SEP]' + df['url'] + '[SEP]' + df['categories'] 
    return df

In [ ]:
def make_eval_ds(df,save_fn="df_eval.pkl"):
    df_eval_g=df.groupby('point_of_interest').size()
    df_eval_g_small=df_eval_g.sort_values(ascending=False)[:12].copy()
    print(df_eval_g_small)
    df_eval_idx=df_eval_g_small.keys()
    print('df_evall_idx: ',df_eval_idx)
    df_eval=df[df['point_of_interest'].isin(df_eval_idx)].copy()
    print(len(df_eval))
    with open(save_fn, 'wb') as f:
        pickle.dump(df_eval, f)
    return df_eval,df_eval_idx
def get_eval(fn):
    with open(fn, 'rb') as f:
        df=pickle.load(f)
    df_eval_g=df.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    return df,df_eval_idx

In [ ]:
def make_train_supplement():
    df = pd.read_csv(input_path+'/'+CFG.train)
    df = make_text(df.fillna("").reset_index())
    df[['index','id','text','point_of_interest']].to_csv(output_path+'/'+CFG.sup_train, index=False)

def get_train_supplement( fn ):
    df = pd.read_csv(fn)
    return df

if not IS_Fig_Only:
    make_train_supplement()

## Train

### datasets

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample,losses
from torch.utils.data import DataLoader
from tqdm import tqdm

if IS_V3_train:
    train_examples = []
    df_train=get_train_supplement()
  
    df_train_g=df_train.groupby('point_of_interest').size()
    df_train_idx=df_train_g.keys()
    dict_train = {df_train_idx[i]: i for i in range(len(df_train_idx))}

    l_df=len(df_train)
    for idx in tqdm(range(l_df)):
        _label=dict_train[df_train.loc[idx,'point_of_interest']]
        train_examples.append(InputExample(texts=[df_train.loc[idx,'text']], label=_label))

### train

In [ ]:
class CFG_V3:
    model_name='xlm-roberta-base'
    batch_size=24
    epochs=1
    pickle_name='/xlm-roberta-base-label.pkl'

In [ ]:
if IS_V3_train:
    model = sentence_transformers.SentenceTransformer(CFG_V3.model_name,device=CFG.device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG_V3.batch_size)
    train_loss = losses.BatchAllTripletLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=CFG_V3.epochs, warmup_steps=100)

    with open(output_path+CFG_V3.pickle_name, 'wb') as f:
        pickle.dump(model, f)
else:
    if not IS_Fig_Only:
        with open(ds_path+CFG_V3.pickle_name, 'rb') as f:
            model=pickle.load( f )

## eval

In [ ]:
if not IS_Fig_Only:
    df_train=get_train_supplement(output_path+'/'+CFG.sup_train)
    df_eval, df_eval_idx = make_eval_ds(df_train)

    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    sentences = df_eval['text'].tolist()

    embeddings = model.encode(sentences)
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())

### post trained fig

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig3post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig3post.png')

多分、xlm-robertaはそのままでは、うまく学習できないかもしれない。

# V4 sentence transformers original

In [ ]:
class CFG_V4:
    model_name='paraphrase-multilingual-MiniLM-L12-v2'
    pkl_name='/paraphrase-multilingual-MiniLM-L12-v2-pair.pkl'

## dataset

In [ ]:
if not IS_Fig_Only:
    df_train=get_train_supplement(output_path+'/'+CFG.sup_train)
    df_eval, df_eval_idx = make_eval_ds(df_train)

    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    sentences = df_eval['text'].tolist()

## pretrained  eval

In [ ]:
if not IS_Fig_Only:
    model = sentence_transformers.SentenceTransformer(CFG_V4.model_name,device=CFG.device)

    embeddings = model.encode(sentences)
    print("Sentence:", sentences[0])
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())

### figure

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig4pre.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig4pre.png')

## Train

### datasets

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample,losses
from torch.utils.data import DataLoader
from tqdm import tqdm

if IS_V4_train:
    train_examples = []
    df_pairs=pd.read_csv(input_path+'/'+CFG.pairs)

    df_p=df_pairs.fillna('')
    df_p['text_1']= df_p['latitude_1'].astype(str)+ '[SEP]'+df_p['longitude_1'].astype(str)+ '[SEP]'+df_p['name_1'] + '[SEP]' + df_p['address_1'] + '[SEP]' + df_p['city_1'] + '[SEP]' + df_p['state_1'] + '[SEP]' + df_p['country_1'] + '[SEP]' + df_p['url_1'] + '[SEP]' + df_p['categories_1'] 
    df_p['text_2']=  df_p['latitude_2'].astype(str)+ '[SEP]'+df_p['longitude_2'].astype(str)+ '[SEP]'+df_p['name_2'] + '[SEP]' + df_p['address_2'] + '[SEP]' + df_p['city_2'] + '[SEP]' + df_p['state_2'] + '[SEP]' + df_p['country_2'] + '[SEP]' + df_p['url_2'] + '[SEP]' + df_p['categories_2'] 

    l_df=len(df_p)

    for idx in tqdm(range(l_df)):
        tof=-0
        if df_p.loc[idx,'match']:
            tof=1
        train_examples.append(InputExample(texts=[df_p.loc[idx,'text_1'],df_p.loc[idx,'text_2']], label=tof))

### train

In [ ]:
if IS_V4_train:
    model = sentence_transformers.SentenceTransformer(CFG_V4.model_name,device=CFG.device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    train_loss = losses.ContrastiveLoss(model)
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

    with open(output_path+CFG_V4.pkl_name, 'wb') as f:
        pickle.dump(model, f)
else:
    if not IS_Fig_Only:
        with open(ds_path+CFG_V4.pkl_name, 'rb') as f:
            model=pickle.load( f )

### post trained fig

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig4post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    embeddings = model.encode(sentences)
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig4post.png')

# V5 next labeled train

In [ ]:
class CFG_V5:
    model_name='paraphrase-multilingual-MiniLM-L12-v2'
    pkl_name='/paraphrase-multilingual-MiniLM-L12-v2-label.pkl'
    batch_size=32
    epochs=1

## dataset

In [ ]:
if not IS_Fig_Only:
    df_train=get_train_supplement(output_path+'/'+CFG.sup_train)
    df_eval, df_eval_idx = make_eval_ds(df_train)

    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    sentences = df_eval['text'].tolist()

## pretrained  eval

In [ ]:
if not IS_Fig_Only:
    model = sentence_transformers.SentenceTransformer(CFG_V5.model_name,device=CFG.device)

    embeddings = model.encode(sentences)
    print("Sentence:", sentences[0])
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())

### figure

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig5pre.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig5pre.png')

## Train

### datasets

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample,losses
from torch.utils.data import DataLoader
from tqdm import tqdm

if IS_V5_train:
    train_examples = []
    df_train=get_train_supplement()
  
    df_train_g=df_train.groupby('point_of_interest').size()
    df_train_idx=df_train_g.keys()
    dict_train = {df_train_idx[i]: i for i in range(len(df_train_idx))}

    l_df=len(df_train)
    for idx in tqdm(range(l_df)):
        _label=dict_train[df_train.loc[idx,'point_of_interest']]
        train_examples.append(InputExample(texts=[df_train.loc[idx,'text']], label=_label))

### train

In [ ]:
if IS_V5_train:
    model = sentence_transformers.SentenceTransformer(CFG_V5.model_name,device=CFG.device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG_V5.batch_size)
    train_loss = losses.BatchAllTripletLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=CFG_V5.epochs, warmup_steps=100)

    with open(output_path+CFG_V5.pkl_name, 'wb') as f:
        pickle.dump(model, f)
else:
    if not IS_Fig_Only:
        with open(ds_path+CFG_V5.pkl_name, 'rb') as f:
            model=pickle.load( f )

### post trained fig

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig5post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)  
else:
    embeddings = model.encode(sentences)
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig5post.png')

# V6 next labeled train epoch5

In [ ]:
class CFG_V6:
    model_name='paraphrase-multilingual-MiniLM-L12-v2'
    pkl_name='/paraphrase-multilingual-MiniLM-L12-v2-label-ep5.pkl'
    batch_size=64
    epochs=5

## dataset

In [ ]:
if not IS_Fig_Only:
    df_train=get_train_supplement(output_path+'/'+CFG.sup_train)
    df_eval, df_eval_idx = make_eval_ds(df_train)

    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    sentences = df_eval['text'].tolist()

## pretrained  eval

In [ ]:
if not IS_Fig_Only:
    model = sentence_transformers.SentenceTransformer(CFG_V5.model_name,device=CFG.device)

    embeddings = model.encode(sentences)
    print("Sentence:", sentences[0])
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())

### figure

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig6pre.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig6pre.png')

## Train

### datasets

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample,losses
from torch.utils.data import DataLoader
from tqdm import tqdm

if IS_V6_train:
    train_examples = []
    df_train=get_train_supplement()

    df_train_g=df_train.groupby('point_of_interest').size()
    df_train_idx=df_train_g.keys()
    dict_train = {df_train_idx[i]: i for i in range(len(df_train_idx))}

    l_df=len(df_train)
    for idx in tqdm(range(l_df)):
        _label=dict_train[df_train.loc[idx,'point_of_interest']]
        train_examples.append(InputExample(texts=[df_train.loc[idx,'text']], label=_label))

### train

In [ ]:
if IS_V6_train:
    model = sentence_transformers.SentenceTransformer(CFG_V6.model_name,device=CFG.device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG_V6.batch_size)
    train_loss = losses.BatchAllTripletLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=CFG_V6.epochs, warmup_steps=100)

    with open(output_path+CFG_V6.pkl_name, 'wb') as f:
        pickle.dump(model, f)
else:
    if not IS_Fig_Only:
        with open(ds_path+CFG_V6.pkl_name, 'rb') as f:
            model=pickle.load( f )


### post trained fig

In [ ]:
if IS_Fig_Only:
    obj=plt.imread(ds_fig_path+'/fig6post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)
else:
    embeddings = model.encode(sentences)
    print("Embedding 12:", embeddings[0][:12])
    p=np.array(embeddings)
    p2=[np.linalg.norm(np.array(p[i]), ord=2) for i in range(len(p))]
    print(np.array(p2).mean())
    print(np.array(p2).max())
    print(np.array(p2).min())
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig6post.png')

# V7 train data

## make small dataset

In [ ]:
df_train=get_train_supplement(ds2_path+'/sup_train.csv')
print('train total data: ',len(df_train))

In [ ]:
def make_train_small(df,save_fn="df_train_s.pkl"):
    df_train_g=df.groupby('point_of_interest').size()
    df_train_g_small=df_train_g.sort_values(ascending=False)[1300:1312].copy()
    print(df_train_g_small)
    df_train_idx=df_train_g_small.keys()
    print('df_train_idx: ',df_train_idx)
    df_train_s=df[df['point_of_interest'].isin(df_train_idx)].copy()
    print(len(df_train_s))
    df_train_s=df_train_s.drop('index',axis=1).reset_index()
    df_train_s=df_train_s.drop('index',axis=1).reset_index()
    with open(save_fn, 'wb') as f:
        pickle.dump(df_train_s, f)
    return df_train_s,df_train_idx

In [ ]:
df_train_s,df_train_idx=make_train_small(df_train)

## K-fold

### groupkfold

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold

def get_poi_groups(train: pd.DataFrame):
    le = LabelEncoder()
    le.fit(train['point_of_interest'])
    groups = le.transform(train['point_of_interest'])
    return groups

groups = get_poi_groups(df_train_s)

group_kf = GroupKFold(n_splits=4) 
for i, (tr_idx, va_idx) in enumerate(group_kf.split(df_train_s,y=None,groups=groups)):
    tra_X, val_X = df_train_s.iloc[tr_idx], df_train_s.iloc[va_idx]
    #処理を書く

In [ ]:
tra_X.groupby('point_of_interest').size()

### k-fold+shuffle

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4,random_state=0, shuffle=True) 


kfr=kf.split(df_train_idx)
print(kfr)

df=pd.DataFrame(df_train_idx)

for i, (tr_idx, va_idx) in enumerate(kf.split(df_train_idx)):
    print(tr_idx)
    print(va_idx)
    df.loc[va_idx,'fold']=i
df=df.astype({'fold': 'int'})

In [ ]:
df_train_s1=df_train_s.merge(df,on=['point_of_interest'],how='left')
df_train_s1

## train dataset for TripletLoss

同じpoiのリストと近すぎる違うpoiのリストを作る

In [ ]:
df_train_g=df_train_s1.groupby('point_of_interest').groups
df_train_g

In [ ]:
class CFG_V7:
    model_name='paraphrase-multilingual-MiniLM-L12-v2'
    batch_size=24
    epochs=1
    pickle_name='/paraphrase-multilingual-MiniLM-L12-v2.pkl'


In [ ]:
model = sentence_transformers.SentenceTransformer(CFG_V7.model_name,device=CFG.device)
sentences = df_train_s['text'].tolist()
embeddings=model.encode(sentences)

## embeddings normalize

In [ ]:
emb=np.array(embeddings)
del embeddings
gc.collect()
emb_n=np.array([emb[i]/np.linalg.norm(emb[i], ord=2) for i in range(len(emb))])
del emb
gc.collect()

## cosine similarity

In [ ]:
list_n=[]
for idx in tqdm(range(len(emb_n))):
    b_emb=emb_n[idx]
    c_s=np.dot(emb_n,b_emb.T).T
    poi=df_train_s.loc[idx,'point_of_interest']
    group_poi=df_train_g[poi]
    c_s[group_poi]=0
    over_c_s=np.where(c_s>0.95)
    list_n.append(over_c_s)
#list_n

## datasets

In [ ]:
df_train_s,df_train_idx=make_train_small(df_train)

df_train_g=df_train_s.groupby('point_of_interest').groups

model = sentence_transformers.SentenceTransformer(CFG_V7.model_name,device=CFG.device)

sentences = df_train_s['text'].tolist()

embeddings=model.encode(sentences)

emb=np.array(embeddings)
del embeddings
gc.collect()
emb_n=np.array([emb[i]/np.linalg.norm(emb[i], ord=2) for i in range(len(emb))])
del emb
gc.collect()

list_ds=[]
for idx in tqdm(range(len(emb_n))):
    b_emb=emb_n[idx]
    c_s=np.dot(emb_n,b_emb.T).T
    poi=df_train_s.loc[idx,'point_of_interest']
    group_poi=df_train_g[poi]
    c_s[group_poi]=0
    over_c_s=np.where(c_s>0.95)
#    print(len(over_c_s[0]))
    if len(over_c_s[0])==0:
        for p_idx in group_poi:
            list_ds.append([idx,p_idx,idx])
    else:
        for p_idx in group_poi:
            for n_idx in over_c_s[0]:
                list_ds.append([idx,p_idx,n_idx])
    


## Train

In [ ]:
IS_V7_train=True

### datasets

In [ ]:
from tqdm import tqdm

if IS_V7_train:
    train_examples = []

    df_train_s,df_train_idx=make_train_small(df_train)

    df_train_g=df_train_s.groupby('point_of_interest').groups

    model = sentence_transformers.SentenceTransformer(CFG_V7.model_name,device=CFG.device)

    sentences = df_train_s['text'].tolist()

    embeddings=model.encode(sentences)

    emb=np.array(embeddings)
    del embeddings
    gc.collect()
    emb_n=np.array([emb[i]/np.linalg.norm(emb[i], ord=2) for i in range(len(emb))])
    del emb
    gc.collect()

    list_ds=[]
    for idx in tqdm(range(len(emb_n))):
        b_emb=emb_n[idx]
        c_s=np.dot(emb_n,b_emb.T).T
        poi=df_train_s.loc[idx,'point_of_interest']
        group_poi=df_train_g[poi]
        c_s[group_poi]=0
        over_c_s=np.where(c_s>0.95)
        if len(over_c_s[0])==0:
            for p_idx in group_poi:
                list_ds.append([idx,p_idx,idx])
        else:
            for p_idx in group_poi:
                for n_idx in over_c_s[0]:
                    list_ds.append([idx,p_idx,n_idx])




#  df_train=get_train_supplement()
  
#  df_train_g=df_train.groupby('point_of_interest').size()
#  df_train_idx=df_train_g.keys()
#  dict_train = {df_train_idx[i]: i for i in range(len(df_train_idx))}
    from sentence_transformers import SentenceTransformer, InputExample,losses
    from torch.utils.data import DataLoader

    l_df=len(list_ds)

    for idx in tqdm(range(l_df)):
        train_examples.append(InputExample(texts=[df_train_s.loc[list_ds[idx][0],'text'],df_train_s.loc[list_ds[idx][1],'text'],df_train_s.loc[list_ds[idx][2],'text']]))

### train

In [ ]:
if IS_V7_train:
    model = sentence_transformers.SentenceTransformer(CFG_V7.model_name,device=CFG.device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG_V7.batch_size)
    train_loss = losses.TripletLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=CFG_V7.epochs, warmup_steps=100)

    with open(output_path+CFG_V7.pickle_name, 'wb') as f:
        pickle.dump(model, f)
else:
    if not IS_Fig_Only:
        with open(ds_path+CFG_V7.pickle_name, 'rb') as f:
            model=pickle.load( f )
